# Exploring Shastry-Sutterland model with RBM variational wave function and other models

In [102]:
import netket as nk
import numpy as np
import time
import json
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import jax
import flax
import optax
from sys import version as pyvers

print("Python version: {}".format(pyvers))
print("NetKet version: {}".format(nk.__version__))
print("NumPy version: {}".format(np.__version__))

Python version: 3.8.10 (default, Nov 26 2021, 20:14:08) 
[GCC 9.3.0]
NetKet version: 3.3.2.post1
NumPy version: 1.20.3


### Setup relevant parameters and settings of the simulation 

In [103]:
"""lattice"""
SITES    = 16            # 4, 8, 16, 20 ... number of vertices in a tile determines the tile shape 
JEXCH1   = .2            # nn interaction
JEXCH2   = 1             # nnn interaction
H_Z      = 1.3             # magnetic field
TOTAL_SZ = None          # 0, None ... restriction of Hilbert space
#USE_MSR = True          # Should we use a Marshall sign rule? In this notebook, we use both.

"""machine learning"""
MACHINE  = "myRBM"         # RBM, RBMSymm, RBMSymm_transl, RBMModPhase, GCNN, Jastrow, myRBM
DTYPE    = np.complex128 # data-type of weights in neural network
ALPHA    = 8            # N_hidden / N_visible
ETA      = .01          # learning rate (0.01 usually works)
SAMPLER  = 'exact'       # 'local' = MetropolisLocal, 'exchange' = MetropolisExchange
SAMPLES  = 1000          # number of Monte Carlo samples
NUM_ITER = 400           # number of convergence iterations
N_LAYERS = 3 #2             # number of layers (in case of G-CNN)
FEATURES = (8,4,4) #16 #(8,4)         # dimensions of layers (in case of G-CNN)

OUT_NAME = "SS_"+str(SITES)+"j1="+str(JEXCH1) # output file name

### Lattice definition
Basic structure of tiled lattices is implemented in `lattice_and_ops.py` file. Class `Lattice` implements relative positional functions, 
- e.g. *`rt(node)` returns the index of the right neighbour of a site with index `node`*

The `for` loop constructs full Shastry-Sutherland lattice with PBC using these auxiliary positional functions.

In [104]:
from lattice_and_ops import Lattice
lattice = Lattice(SITES)

# Construction of custom graph according to tiled lattice structure defined in the Lattice class.
edge_colors = []
for node in range(SITES):
    edge_colors.append([node,lattice.rt(node), 1])  # horizontal connections
    edge_colors.append([node,lattice.bot(node), 1]) # vertical connections
    row, column = lattice.position(node)
    if column%2 == 0:
        if row%2 == 0:
            edge_colors.append([node,lattice.lrt(node),2]) # diagonal bond
        else:
            edge_colors.append([node,lattice.llft(node),2]) # diagonal bond

g = nk.graph.Graph(edges=edge_colors) #,n_nodes=3)
N = g.n_nodes

hilbert = nk.hilbert.Spin(s=.5, N=g.n_nodes, total_sz=TOTAL_SZ)

In [105]:
"""alternative (simple) toy lattices"""
# # two dimers
# edge_colors.append([0,1,1])
# edge_colors.append([2,3,1])

# # SS 2x2 lattice:
# edge_colors.append([0,1,1])
# edge_colors.append([2,3,1])
# edge_colors.append([1,2,1])
# edge_colors.append([0,3,1])
# edge_colors.append([0,2,2])
# edge_colors.append([1,3,2])

# # 3-chain:
# edge_colors.append([0,1,1])
# edge_colors.append([1,2,1])
# edge_colors.append([2,0,1])

'alternative (simple) toy lattices'

### Characters of the symmetries
In case of G-CNN, we need to specify the characters of the symmetry transformations.
- DS phase anti-symmetric wrt permutations with negative sign and symmetric wer permutaions with postive sign
- AF phase is always symmetric for all permutations  

In [106]:

print("There are", len(g.automorphisms()), "full symmetries.")
# deciding point between DS and AF phase is set to 0.5
if JEXCH1 < 0.5:
    # DS phase is partly anti-symmetric
    characters = []
    from lattice_and_ops import permutation_sign
    for perm in g.automorphisms():
        # print(perm, "with sign", permutation_sign(np.asarray(perm)))
        characters.append(permutation_sign(np.asarray(perm)))
    characters_dimer = np.asarray(characters,dtype=complex)
    characters_dimer_msr = characters_dimer
else:
    # AF phase if fully symmetric
    characters_dimer = np.ones((len(g.automorphisms()),), dtype=complex)
    characters_dimer_msr = characters_dimer

There are 64 full symmetries.


### Translations

If we want to include only translations, we have to exclude some symmetries from `g.automorphisms()`.


⚠️ TODO ⚠️ <span style="color:red"> This part is not fully automated yet. Translations are currently picked by hand from the group of all automorphisms. </span>

In [107]:
if MACHINE == "RBMSymm_transl" and N != 4:
    raise NotImplementedError("Extraction of translations from the group of automorphisms is not implemented yet.")
translations = []
for perm in g.automorphisms():
    aperm = np.asarray(perm)
    if (aperm[0],aperm[1]) in ((0,1),(1,0),(2,3),(3,2)): # N = 4
    # if (aperm[0],aperm[1],aperm[3]) in ((0,1,3),(2,3,1),(8,9,11),(10,11,9)): # N = 16
    # if (aperm[0],aperm[1],aperm[2],aperm[3]) in ((4,7,6,5),): # N = 8
    # if (aperm[2],aperm[0]) in ((2,0),(3,0),(0,1),(1,2)):#,(2,3,1)): # N = 16, two dimers with just translation
    # if (aperm[0],aperm[1],aperm[3]) in ((0,1,3),(3,2,0),(2,3,1),(1,0,2)): # N = 16, two dimers plus second translation option
        translations.append(nk.utils.group._permutation_group.Permutation(aperm))
translation_group = nk.utils.group._permutation_group.PermutationGroup(translations,degree=SITES)
print("Out of", len(g.automorphisms()), "permutations,",len(translation_group), "translations were picked.")

Out of 64 permutations, 4 translations were picked.


In [108]:
g.automorphisms().character_table_readable()

(['1xPermutation([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])',
  '4xPermutation([0, 1, 13, 12, 4, 5, 9, 8, 7, 6, 10, 11, 3, 2, 14, 15])',
  '4xPermutation([0, 4, 8, 12, 1, 5, 9, 13, 2, 6, 10, 14, 3, 7, 11, 15])',
  '8xPermutation([1, 2, 3, 0, 13, 14, 15, 12, 9, 10, 11, 8, 5, 6, 7, 4])',
  '8xPermutation([1, 2, 6, 5, 13, 14, 10, 9, 12, 15, 11, 8, 0, 3, 7, 4])',
  '4xPermutation([2, 3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12, 13])',
  '8xPermutation([2, 3, 15, 14, 6, 7, 11, 10, 5, 4, 8, 9, 1, 0, 12, 13])',
  '4xPermutation([2, 6, 10, 14, 3, 7, 11, 15, 0, 4, 8, 12, 1, 5, 9, 13])',
  '8xPermutation([3, 0, 1, 2, 15, 12, 13, 14, 11, 8, 9, 10, 7, 4, 5, 6])',
  '8xPermutation([3, 0, 4, 7, 15, 12, 8, 11, 14, 13, 9, 10, 2, 1, 5, 6])',
  '2xPermutation([5, 4, 7, 6, 1, 0, 3, 2, 13, 12, 15, 14, 9, 8, 11, 10])',
  '4xPermutation([5, 4, 8, 9, 1, 0, 12, 13, 2, 3, 15, 14, 6, 7, 11, 10])',
  '1xPermutation([10, 11, 8, 9, 14, 15, 12, 13, 2, 3, 0, 1, 6, 7, 4, 5])'],
 array([[ 1.+0.j,  1.+0.

## Hamoltonian definition
$$ H = J_{1} \sum\limits_{\langle i,j \rangle}^{L} \vec{\sigma}_{i} \cdot \vec{\sigma}_{j} + J_{2} \sum\limits_{\langle\langle i,j \rangle\rangle_{SS}}^{L}  \vec{\sigma}_{i} \cdot \vec{\sigma}_{j} + 2h\sum\limits_{i} \sigma^z_{i}\,. $$

Axiliary constant operators used to define hamiltonian are loaded from the external file, they are pre-defined in the `HamOps` class.

In [109]:
from lattice_and_ops import HamOps
ho = HamOps()
# H_Z = .5
ha_1 = nk.operator.GraphOperator(hilbert, graph=g, bond_ops=ho.bond_operator(JEXCH1,JEXCH2, h_z=H_Z, use_MSR=False), bond_ops_colors=ho.bond_color)
ha_2 = nk.operator.GraphOperator(hilbert, graph=g, bond_ops=ho.bond_operator(JEXCH1,JEXCH2, h_z=H_Z, use_MSR=True), bond_ops_colors=ho.bond_color)


In [110]:
print(ha_2.to_sparse()[0])
# print(ha_2.to_sparse())

  (0, 0)	56.00000000000004


### Magnetization operator definition
$$ \hat{m}_z := \sum\limits_i \sigma_i^z $$

In [111]:
m_z = sum(nk.operator.spin.sigmaz(hilbert, i) for i in range(hilbert.size))

In [112]:
if False: # exact diagonalization scan
    print("#N =", N)
    print("# J1     J2      H_Z            m_z            ground          1.excited          2.excited")
    for H_Z in np.arange(1.37,1.45,step=0.01):
        ha = nk.operator.GraphOperator(hilbert, graph=g, bond_ops=ho.bond_operator(JEXCH1,JEXCH2, h_z=H_Z, use_MSR=False), bond_ops_colors=ho.bond_color)
        start = time.time()
        evals, eigvects = nk.exact.lanczos_ed(ha, k=3, compute_eigenvectors=True)
        ground_state = eigvects.T[0]
        end = time.time()
        magnetization_z = ground_state.transpose()@(m_z@ground_state)
        # print("ED took", (start-end)/60, "min.")
        # print(np.round(np.array(ground_state),5))
        # print(ha.to_dense())
        print("%.3f   %.3f  %2.10f   %2.10f   %2.10f   %14.10f   %14.10f" % (JEXCH1, JEXCH2, H_Z, magnetization_z, evals[0], evals[1], evals[2]))

## Exact diagonalization

In [113]:
if g.n_nodes < 20:
    start = time.time()
    if g.n_nodes < 15:
        evals, eigvects = nk.exact.full_ed(ha_1, compute_eigenvectors=True)
    else:
        evals, eigvects = nk.exact.lanczos_ed(ha_1, k=3, compute_eigenvectors=True)
    end = time.time()
    diag_time = end - start
    print("Ground state energy:",evals[0], "\nIt took ", round(diag_time,2), "s =", round((diag_time)/60,2),"min")
else:
    print("System is too large for exact diagonalization. Setting exact_ground_energy = 0 (which is wrong)")
    evals = [0,0,0]
    eigvects = None 
exact_ground_energy = evals[0]

Ground state energy: -29.04857609095167 
It took  4.54 s = 0.08 min


# Machine learning starts here

## Machine definition and other auxiliary `netket` objects
We define two sets of these objects, usually: 
- variables ending with ...`_1` belongs to the choice of standard basis,
- variables ending with ...`_2` belongs to the choice of MSR basis.

But they can be used in a different way when we need to compare two different models.

In [114]:
# Selection of machine type
if MACHINE == "RBM":
    machine_1 = nk.models.RBM(dtype=DTYPE, alpha=ALPHA)#, use_visible_bias=False) 
    machine_2 = nk.models.RBM(dtype=DTYPE, alpha=ALPHA)#, use_visible_bias=False)
elif MACHINE == "RBMSymm":
    machine_1 = nk.models.RBMSymm(g.automorphisms(), dtype=DTYPE, alpha=ALPHA)#, use_visible_bias=False) 
    machine_2 = nk.models.RBMSymm(g.automorphisms(), dtype=DTYPE, alpha=ALPHA)#, use_visible_bias=False)
elif MACHINE == "RBMSymm_transl":
    machine_1 = nk.models.RBMSymm(translation_group, dtype=DTYPE, alpha=ALPHA)#, use_visible_bias=False) 
    machine_2 = nk.models.RBMSymm(translation_group, dtype=DTYPE, alpha=ALPHA)#, use_visible_bias=False)
elif MACHINE == "GCNN":
    machine_1 = nk.models.GCNN(symmetries=g.automorphisms(), dtype=DTYPE, 
        layers=N_LAYERS, features=FEATURES, characters=characters_dimer    )#, output_activation=log_cosh)
    machine_2 = nk.models.GCNN(symmetries=g.automorphisms(), dtype=DTYPE, 
        layers=N_LAYERS, features=FEATURES, characters=characters_dimer_msr)#, output_activation=log_cosh)
elif MACHINE == "myRBM":
        from GCNN_Nomura import GCNN_my
        machine_1 = GCNN_my(symmetries=g.automorphisms(), dtype=fq.DTYPE, layers=1, features=fq.ALPHA, characters=characters_dimer_1, output_activation=nk.nn.log_cosh, use_bias=True, use_visible_bias=True)
        machine_2 = GCNN_my(symmetries=g.automorphisms(), dtype=fq.DTYPE, layers=1, features=fq.ALPHA, characters=characters_dimer_2, output_activation=nk.nn.log_cosh, use_bias=True, use_visible_bias=True)
elif MACHINE == "Jastrow":
    from lattice_and_ops import Jastrow
    machine_1 = Jastrow()
    machine_2 = Jastrow()
elif MACHINE == "RBMModPhase":
    machine_1 = nk.models.RBMModPhase(alpha=ALPHA, use_hidden_bias=True, dtype=DTYPE)
    machine_2 = nk.models.RBMModPhase(alpha=ALPHA, use_hidden_bias=True, dtype=DTYPE)

    # A linear schedule varies the learning rate from 0 to 0.01 across 600 steps.
    modulus_schedule_1=optax.linear_schedule(0,0.01,NUM_ITER)
    modulus_schedule_2=optax.linear_schedule(0,0.01,NUM_ITER)
    # The phase starts with a larger learning rate and then is decreased.
    phase_schedule_1=optax.linear_schedule(0.05,0.01,NUM_ITER)
    phase_schedule_2=optax.linear_schedule(0.05,0.01,NUM_ITER)
    # Combine the linear schedule with SGD
    optm_1=optax.sgd(modulus_schedule_1)
    optp_1=optax.sgd(phase_schedule_1)
    optm_2=optax.sgd(modulus_schedule_2)
    optp_2=optax.sgd(phase_schedule_2)
    # The multi-transform optimizer uses different optimisers for different parts of the parameters.
    optimizer_1 = optax.multi_transform({'o1': optm_1, 'o2': optp_1}, flax.core.freeze({"Dense_0":"o1", "Dense_1":"o2"}))
    optimizer_2 = optax.multi_transform({'o1': optm_2, 'o2': optp_2}, flax.core.freeze({"Dense_0":"o1", "Dense_1":"o2"}))
else:
    raise Exception(str("undefined MACHINE: ")+str(MACHINE))

# Selection of sampler type
if SAMPLER == 'local':
    sampler_1 = nk.sampler.MetropolisLocal(hilbert=hilbert)
    sampler_2 = nk.sampler.MetropolisLocal(hilbert=hilbert)
elif SAMPLER == 'exact':
    sampler_1 = nk.sampler.ExactSampler(hilbert=hilbert)
    sampler_2 = nk.sampler.ExactSampler(hilbert=hilbert)
else:
    sampler_1 = nk.sampler.MetropolisExchange(hilbert=hilbert, graph=g)
    sampler_2 = nk.sampler.MetropolisExchange(hilbert=hilbert, graph=g)
    if SAMPLER != 'exchange':
        print("Warning! Undefined fq.SAMPLER:", SAMPLER, ", dafaulting to MetropolisExchange fq.SAMPLER")

if MACHINE != "RBMModPhase":
    optimizer_1 = nk.optimizer.Sgd(learning_rate=ETA)
    optimizer_2 = nk.optimizer.Sgd(learning_rate=ETA)

# Stochastic Reconfiguration as a preconditioner
sr_1  = nk.optimizer.SR(diag_shift=0.01)
sr_2  = nk.optimizer.SR(diag_shift=0.01)

# The variational state (former name: nk.variational.MCState)
vs_1 = nk.vqs.MCState(sampler_1 , machine_1 , n_samples=SAMPLES)
vs_2 = nk.vqs.MCState(sampler_2 , machine_2 , n_samples=SAMPLES)
vs_1.init_parameters(jax.nn.initializers.normal(stddev=0.001))
vs_2.init_parameters(jax.nn.initializers.normal(stddev=0.001))


gs_1 = nk.VMC(hamiltonian=ha_1 ,optimizer=optimizer_1 ,preconditioner=sr_1 ,variational_state=vs_1)
gs_2 = nk.VMC(hamiltonian=ha_2 ,optimizer=optimizer_2 ,preconditioner=sr_2 ,variational_state=vs_2) 

Exception: Invalid ALPHA. It needs to be divisible by the number of symmetries!

In [ ]:
print(vs_1.n_parameters, vs_2.n_parameters)

4368 4368


# Calculation
We let the calculation run for `NUM_ITERS` iterations for both cases _1 and _2 (without MSR and with MSR). If only one case is desired, set the variable `no_of_runs` to 1.

In [ ]:
runs = [1,1]
no_of_runs = np.sum(runs) # 1 - one run for variables with ..._1;  2 - both runs for variables ..._1 and ..._2
run_only_2 = (runs[1]==1 and runs[0]==0) # in case of no_of_runs=1
# NUM_ITER = 100#3000
print("J_1 =", JEXCH1, end="; ")
if exact_ground_energy != 0:
    print("Expected exact energy:", exact_ground_energy)
for i,gs in enumerate([gs_1,gs_2][run_only_2:run_only_2+no_of_runs]):
    start = time.time()
    gs.run(out=OUT_NAME+str(i), n_iter=int(NUM_ITER))#, obs={'symmetry':P(0,1)})
    end = time.time()
    print("The calculation for {} of type {} took {} min".format(MACHINE, i+1, (end-start)/60))


J_1 = 0.2; Expected exact energy: -29.04857609095212


  0%|          | 0/400 [00:00<?, ?it/s]

x=Traced<ShapedArray(complex128[256])>with<DynamicJaxprTrace(level=0/1)>  target=Traced<ShapedArray(complex128[256])>with<DynamicJaxprTrace(level=0/1)>
x=Traced<ShapedArray(complex128[16,256])>with<DynamicJaxprTrace(level=0/1)>  target=Traced<ShapedArray(complex128[16,256])>with<DynamicJaxprTrace(level=0/1)>
x=Traced<ShapedArray(complex128[16])>with<DynamicJaxprTrace(level=0/1)>  target=Traced<ShapedArray(complex128[16])>with<DynamicJaxprTrace(level=0/1)>
x=Traced<ShapedArray(complex128[256])>with<DynamicJaxprTrace(level=1/2)>  target=Traced<ShapedArray(complex128[256])>with<DynamicJaxprTrace(level=1/2)>
x=Traced<ShapedArray(complex128[16,256])>with<DynamicJaxprTrace(level=1/2)>  target=Traced<ShapedArray(complex128[16,256])>with<DynamicJaxprTrace(level=1/2)>
x=Traced<ShapedArray(complex128[16])>with<DynamicJaxprTrace(level=1/2)>  target=Traced<ShapedArray(complex128[16])>with<DynamicJaxprTrace(level=1/2)>
x=Traced<ShapedArray(complex128[256])>with<DynamicJaxprTrace(level=1/2)>  target

100%|██████████| 400/400 [03:27<00:00,  1.93it/s, Energy=-2.720e+01+0.000e+00j ± 2.512e-15 [σ²=5.049e-29]]


The calculation for RBM of type 1 took 3.5547411402066547 min


100%|██████████| 400/400 [03:52<00:00,  1.72it/s, Energy=-27.1975-0.0007j ± 0.0026 [σ²=0.0067]]           

The calculation for RBM of type 2 took 3.891228894392649 min


In [ ]:
for i,gs in enumerate([gs_1,gs_2][run_only_2:run_only_2+no_of_runs]):
    print("Trained RBM with MSR:" if i else "Trained RBM without MSR:")
    print("Energy =", gs.energy)
    print("m_z =", gs.estimate(m_z))

Trained RBM without MSR:
Energy = -2.720e+01+0.000e+00j ± 2.512e-15 [σ²=5.049e-29]
m_z = -1.600e+01+0.000e+00j ± nan [σ²=0.000e+00]
Trained RBM with MSR:
Energy = -27.1975-0.0007j ± 0.0026 [σ²=0.0067]
m_z = -1.600e+01+0.000e+00j ± nan [σ²=0.000e+00]


## Energy Convergence Plotting
In case that the machine did not converge, we can re-run the previous cell and than skip the next cell. This way, the replotting just appends the new results and does not erase the previoius results. 

In [ ]:
# Exact Energy Line
no_of_all_iters = NUM_ITER
figure = go.Figure(
    data=[
        go.Scatter(
            x=(0,no_of_all_iters),
            y=(exact_ground_energy,exact_ground_energy),
            mode="lines",line=go.scatter.Line(color="#000000",width=1), name="exact energy")],
    layout=go.Layout(
        template="simple_white",
        xaxis=dict(title="Iteration", mirror=True, showline=True),
        yaxis=dict(title="Energy", mirror=True, showline=True),
        title=("<b>"+"S-S"+" model </b>, L="+str(SITES)+", J2 ="+str(JEXCH2)+ ", J1 ="+str(JEXCH1)+" , η="+str(ETA)+", α="+str(ALPHA)+", samples="+str(SAMPLES)))
    ).add_hline(y=exact_ground_energy, opacity=1, line_width=1)


In [ ]:
# import the data from log file
OUT_NAME_suffixless=OUT_NAME
data = []
names = ["normal","MSR"]
for i in range(no_of_runs):
    data.append(json.load(open(OUT_NAME_suffixless+str(i)+".log")))
if type(data[0]["Energy"]["Mean"]) == dict: #DTYPE in (np.complex128, np.complex64):#, np.float64):# and False:
    energy_convergence = [data[i]["Energy"]["Mean"]["real"] for i in range(no_of_runs)]
    # symmetry = [data[i]["symmetry"]["Mean"]["real"] for i in range(no_of_runs-run_only_2)]
else:
    energy_convergence = [data[i]["Energy"]["Mean"] for i in range(no_of_runs)]
    # symmetry = [data[i]["symmetry"]["Mean"] for i in range(no_of_runs-run_only_2)]
for i in range(no_of_runs):
    figure.add_trace(go.Scatter(
        x=data[i]["Energy"]["iters"], y=energy_convergence[i],
        name=names[i]
    ))
    # figure.add_trace(go.Scatter(
    #     x=data[i]["Energy"]["iters"], y=symmetry[i],
    #     name=names[i]+"_swap"
    # ))

figure.update_layout(xaxis_title="Iteration",yaxis_title="Energy")
figure.show()

## Assessment of the other simulation results

In [ ]:
# Calculation of how long it took to reach 99.5% of exact energy. The first value under 0.5% deviation counts as a converged state.
no_of_runs = 2
threshold_energy = 0.995*exact_ground_energy
data = []
for i in range(no_of_runs):
    data.append(json.load(open(OUT_NAME+str(i)+".log")))
if type(data[0]["Energy"]["Mean"]) == dict:
    energy_convergence = [data[i]["Energy"]["Mean"]["real"] for i in range(no_of_runs)]
else:
    energy_convergence = [data[i]["Energy"]["Mean"] for i in range(no_of_runs)]
steps_until_convergence = [next((i for i,v in enumerate(energy_convergence[j]) if v < threshold_energy), -1) for j in range(no_of_runs)]
print(steps_until_convergence)

[-1, -1]


In [ ]:
# Evaluation of order parameters.
from lattice_and_ops import Operators, Lattice
ops = Operators(lattice,hilbert,ho.mszsz,ho.exchange)
for i,gs in enumerate([gs_1,gs_2][run_only_2:run_only_2+no_of_runs]):
    print("Trained RBM with MSR:" if i else "Trained RBM without MSR:")
    print("m_d^2 =", gs.estimate(ops.m_dimer_op))
    print("m_p(MSR) =", gs.estimate(ops.m_plaquette_op_MSR))
    print("m_s^2 =", gs.estimate(ops.m_s2_op_MSR))
    print("m_s^2(MSR) =", gs.estimate(ops.m_s2_op))

Trained RBM without MSR:
m_d^2 = -3.333e-01+0.000e+00j ± 9.813e-18 [σ²=3.081e-33]
m_p(MSR) = 0.000e+00+0.000e+00j ± nan [σ²=0.000e+00]
m_s^2 = 1.250e-01+0.000e+00j ± nan [σ²=0.000e+00]
m_s^2(MSR) = 1.250e-01+0.000e+00j ± nan [σ²=0.000e+00]
Trained RBM with MSR:
m_d^2 = -3.333e-01+0.000e+00j ± 9.813e-18 [σ²=3.081e-33]
m_p(MSR) = 0.000e+00+0.000e+00j ± nan [σ²=0.000e+00]
m_s^2 = 1.250e-01+0.000e+00j ± nan [σ²=0.000e+00]
m_s^2(MSR) = 1.250e-01+0.000e+00j ± nan [σ²=0.000e+00]


### Print final results

In [ ]:
from lattice_and_ops import log_results
log_results(JEXCH1,gs_1,gs_2,ops,SAMPLES,NUM_ITER,steps_until_convergence)

 0.200  -27.20000  0.00000   -27.19752  0.00669  -0.3333  0.0000   0.0000  0.0000   0.1250  0.0000  -0.3333  0.0000   0.0000  0.0000   0.1250  0.0000   1000   400 -1, -1


In [ ]:
print("{:6.3f} {:10.5f} {:8.5f}  {:7.4f} {:7.4f}  {:7.4f} {:7.4f}  {:7.4f} {:7.4f}  {:5.0f} {:5.0f}".format(JEXCH1, 
    gs_1.energy.mean.real,                          gs_1.energy.variance,
    gs_1.estimate(ops.m_dimer_op).mean.real,        gs_1.estimate(ops.m_dimer_op).variance,
    gs_1.estimate(ops.m_plaquette_op).mean.real,    gs_1.estimate(ops.m_plaquette_op).variance,
    gs_1.estimate(ops.m_s2_op).mean.real,           gs_1.estimate(ops.m_s2_op).variance, 
    SAMPLES, NUM_ITER, steps_until_convergence[0], sep='    '))

 0.200  -27.20000  0.00000  -0.3333  0.0000   0.0000  0.0000   0.1250  0.0000   1000   400


In [ ]:
print("asd",file=None)

asd


In [ ]:
print("{:9.5f}".format(gs_1.energy.variance))

  0.00000


In [ ]:
gs_1.energy.to_dict()

{'Mean': (-27.200000000000017+0j),
 'Variance': 5.048709793414476e-29,
 'Sigma': 2.5121479338940403e-15,
 'R_hat': nan,
 'TauCorr': 62.0}